<a href="https://colab.research.google.com/github/NatashaB-randao/NLP/blob/main/classificacao_textos_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook apresenta o desenvolvimento de um projeto de classificação de textos utilizando o modelo BERT. O processo é estruturado e documentado, abrangendo todas as etapas essenciais, desde o carregamento e pré-processamento dos dados até o treinamento, ajuste fino e avaliação do modelo. Um pipeline completo é implementado para garantir a reprodutibilidade e a eficiência do experimento.

### Projeto: Classificação de Textos com BERT
Este projeto utiliza o modelo BERT pré-treinado para classificar textos do dataset AG News em quatro categorias:

- World (Notícias sobre o mundo)
- Sports (Esportes)
- Business (Negócios)
- Sci/Tech (Ciência/Tecnologia)

### Instalação das Dependências

In [2]:
pip install transformers datasets scikit-learn torch

### Importação das Bibliotecas

In [1]:
import numpy as np
import torch
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                        Trainer, TrainingArguments, pipeline)

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments


C:\Users\natas\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Carregamento do Dataset

O AG News é um dataset de notícias já etiquetadas em quatro categorias.
Baixamos o conjunto de dados diretamente do Hugging Face Datasets.

In [2]:
# Carregar dataset AG News
dataset = load_dataset('ag_news')

# Exibir um exemplo do dataset
print(dataset['train'][0])


# OBS.: O campo text contém o conteúdo da notícia, e label indica a categoria (0 a 3).


{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


### Pré-processamento dos Dados

In [3]:
# Inicializando o tokenizer do BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Função de tokenização e preparação dos dados
def preprocess_data(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Aplicando a tokenização ao dataset
tokenized_datasets = dataset.map(preprocess_data, batched=True)

### Carregamento do Modelo BERT

In [4]:
# Carregando o modelo BERT para classificação de sequência
num_classes = 4  # Número de classes no dataset AG News
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Preparação para o Treinamento

In [6]:
# Definindo os argumentos de treinamento
training_args = TrainingArguments(
    output_dir='./results',          # Diretório para salvar os resultados
    num_train_epochs=3,              # Número de épocas de treinamento
    per_device_train_batch_size=16,  # Tamanho do lote de treinamento
    per_device_eval_batch_size=64,   # Tamanho do lote de avaliação
    warmup_steps=500,                 # Número de passos para aquecimento
    weight_decay=0.01,               # Taxa de decaimento de peso
    logging_dir='./logs',            # Diretório para logs
    logging_steps=10,                 # Frequência de logs
    report_to="none",                # Desativar W&B se não for necessário
)

### Criando o Trainer

In [7]:
# Criando o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
)


### Treinamento do Modelo

In [ ]:
# Treinando o modelo
trainer.train()

### Avaliação do Modelo

In [ ]:
# Avaliando o modelo no conjunto de teste
evaluation_results = trainer.evaluate()
print(evaluation_results)


### Teste do Modelo

In [ ]:
# Função para realizar previsões em um texto específico
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    return predictions.item()

# Testando o modelo com um exemplo
test_text = "This is an example news article about technology."
predicted_class = predict(test_text)
print(f'Predição para o texto: {predicted_class}')


### Fazer Previsões em um Conjunto de Dados

In [ ]:
# Fazendo previsões em um conjunto de dados
test_samples = ["Breaking news: Scientists discover new planet.",
                "The economy is experiencing a downturn."]

predictions = [predict(sample) for sample in test_samples]
for sample, pred in zip(test_samples, predictions):
    print(f'Texto: "{sample}" - Predição: {pred}')
